In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import expipe
import pathlib
import numpy as np
import sys
sys.path.append('../mec_theta')
import data_processing as dp
#import septum_mec.analysis.data_processing as dp
from trackunitmulticomparison import TrackMultipleSessions
#from septum_mec.analysis.registration import store_notebook
import re
import joblib
import multiprocessing
import shutil
import psutil
import pandas as pd
import matplotlib.pyplot as plt
import quantities as pq
import exdir
from distutils.dir_util import copy_tree
from datetime import timedelta
from tqdm import tqdm_notebook as tqdm
#from septum_mec.analysis.trackunitmulticomparison import TrackMultipleSessions
import networkx as nx
#from nxpd import draw
%matplotlib inline

In [3]:
project_path = dp.project_path()

project = expipe.get_project(project_path)
actions = project.actions

In [4]:
output = pathlib.Path('output/identify_neurons')
output.mkdir(parents=True, exist_ok=True)

In [5]:
identify_neurons = project.require_action('identify-neurons')

In [6]:
actions

In [8]:
actions['MBR-0013-280820-1'].attributes

{'registered': '2020-08-28T13:36:31',
 'datetime': '2020-08-28T13:16:53',
 'type': 'Recording',
 'tags': ['open-ephys'],
 'entities': ['MBR-0013'],
 'users': ['maria'],
 'location': 'ibv',
 'data': {'main': 'actions\\MBR-0013-280820-1\\data\\main.exdir'}}

In [9]:
data_loader = dp.Data()

In [10]:
skip_actions = []

In [21]:
sessions = []
for action in actions.values():
    if action.id in skip_actions:
        continue
    if action.type != 'Recording':
        continue
    action_data_path = dp.get_data_path(action)
    exdir_object = exdir.File(action_data_path)
    if 'processing' not in exdir_object: 
        continue
        
    processing = exdir_object['processing']
    print(action.id)

    if not 'electrophysiology' in processing:
        continue
    elphys = processing['electrophysiology']
    if 'spikesorting' not in elphys:
        continue

    sessions.append({
        'action': action.id,
        'session': int(action.id.split('-')[-1]),
        'entity': int(action.id.split('-')[1]),
        
    })
sessions = pd.DataFrame(sessions)

MBR-0011-030820-1
MBR-0011-110820-3
MBR-0011-120820-1
MBR-0012-110820-1
MBR-0012-120820-1
MBR-0013-060820-1
MBR-0013-060820-3
MBR-0013-100820-1
MBR-0013-110820-1
MBR-0013-120820-1
MBR-0013-130820-1
MBR-0013-130820-2
MBR-0013-240820-1
MBR-0013-250820-1
MBR-0013-260820-1
MBR-0013-270820-1
MBR-0013-270820-2
MBR-0013-280820-1
MBR-0013-280820-2


In [24]:
sessions['date'] = sessions.apply(lambda x: x.action.split('-')[2], axis=1)
# sessions['entity_date'] = sessions.apply(lambda x: få resten kopiert av sarah

In [25]:
sessions.to_csv(output / 'sessions.csv', index=False)

# Identify unique neurons

In [79]:
include = ['MBR-0013-270820-1',
           'MBR-0013-280820-1'
          ]

In [80]:
sessions_include = sessions.query('action.isin(@include)')

In [81]:
sessions_include

,action,session,entity,date
15,MBR-0013-270820-1,1,13,270820
17,MBR-0013-280820-1,1,13,280820


In [104]:
# save graphs
for entity, values in sessions_include.groupby('entity'):
    data_path = output / f'{entity}-graphs'
    if data_path.exists():
        continue
    print('Processing', entity)
    unit_matching = TrackMultipleSessions(
        actions, values.action.values.tolist(), 
        progress_bar=tqdm, verbose=False, data_path=data_path
    )
    unit_matching.do_matching()
    unit_matching.make_graphs_from_matches()
#     unit_matching.compute_time_delta_edges()
    unit_matching.compute_depth_delta_edges()
    # save graph with all dissimilarities for later use
    unit_matching.save_graphs()

Processing 13


# Plot comparisons

In [121]:
unit_comp = unit_matching

In [116]:
unit_comp = TrackMultipleSessions(actions, data_path=f'output/identify_neurons/13-graphs')

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [117]:
unit_comp.load_graphs()

In [122]:
max_dissimilarity = .05
max_depth_delta = 100

unit_comp.remove_edges_above_threshold('weight', max_dissimilarity)
unit_comp.remove_edges_above_threshold('depth_delta', max_depth_delta)

In [123]:
unit_comp.remove_edges_with_duplicate_actions()
unit_comp.identify_units()

In [124]:
unit_comp.identified_units

{0: {'e21ac486-34f4-488f-9f75-805f67ccb6df': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'MBR-0013-270820-1': ['71']})},
  '6eddf84b-59a8-42de-90e2-c5104dc0c932': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'MBR-0013-280820-1': ['223']})},
  '8431b0e3-363e-49e2-baca-82041972ce1c': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'MBR-0013-280820-1': ['287']})},
  'abb66adc-b7c8-43e5-8b3e-ef44261cb874': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'MBR-0013-280820-1': ['291']})},
  'de518b41-0392-4564-8765-21e41e59a4bd': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'MBR-0013-280820-1': ['293']})},
  '491c498a-a878-4010-8264-3f91aea6e285': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'MBR-0013-280820-1': ['297']})}},
 1: {'f3d0b29a-7639-4a04-958c-d5aa6a8e37b7': {'average_dissimilarity': None,
   'original_unit_id

In [125]:
unit_comp.plot_matches('template', chan_group=0, step_color=False)
plt.tight_layout()

ValueError: figure size must be positive finite not (10, 0)

# Store uniqe unit ids to csv

In [88]:
max_dissimilarity = .05
max_depth_delta = 100
id_nums = {}
num = 0
for entity, values in sessions_include.groupby('entity'):
    data_path = output / f'{entity}-graphs'
    unit_matching = TrackMultipleSessions(
        actions, values.action.values.tolist(), 
        progress_bar=tqdm, verbose=False, data_path=data_path
    )
    unit_matching.load_graphs()
    # cutoff large dissimilarities
    unit_matching.remove_edges_above_threshold('weight', max_dissimilarity)
    unit_matching.remove_edges_above_threshold('depth_delta', max_depth_delta)
    unit_matching.remove_edges_with_duplicate_actions()
    unit_matching.identify_units()
    units = []
    for ch, group in unit_matching.identified_units.items():
        for unit_id, val in group.items():
            for action_id, orig_unit_ids in val['original_unit_ids'].items():
                if unit_id not in id_nums:
                    id_nums[unit_id] = num
                    num += 1
                units.extend([
                    {
                        'unit_name': name, 
                        'unit_idnum': id_nums[unit_id],
                        'unit_id': unit_id, 
                        'action': action_id,
                        'channel_group': ch,
                        'max_dissimilarity': max_dissimilarity,
                        'max_depth_delta': max_depth_delta
                    } 
                    for name in orig_unit_ids])
    units = pd.DataFrame(units)
    units = units[~units.action.isin(skip_actions)]
    units.to_csv(output / f'{entity}-units.csv', index=False)

ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/object/apply:numpy.core.multiarray.scalar'
  in "output\identify_neurons\13-graphs\graph-group-0.yaml", line 36, column 14

In [45]:
unique_units = pd.concat([
    pd.read_csv(p) 
    for p in output.iterdir() 
    if p.name.endswith('-units.csv')])

In [ ]:
# remove 1834-010319-3 unit 72

In [46]:
unique_units.to_csv(output / 'units.csv', index=False)

# Store results in Expipe action

In [47]:
identify_neurons.data['sessions'] = 'sessions.csv'
identify_neurons.data['units'] = 'units.csv'

In [48]:
copy_tree(output, str(identify_neurons.data_path()))

['/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-units.csv',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/all_non_identified_units.csv',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-7.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-0.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-6.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-1.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-2.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-5.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-3.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-4.yaml',
 '/media/storage/expipe/septum-mec/act

In [96]:
store_notebook(
    identify_neurons, "00-identify-neurons.ipynb")